<a href="https://colab.research.google.com/github/purvadanait/Final-Year-Project/blob/main/Crack%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from pathlib import Path
from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
positive_dir = Path(r'/content/drive/My Drive/Positive')
negative_dir = Path(r'/content/drive/My Drive/Negative')

In [ ]:
!ls "/content/drive/My Drive/Positive"

 00001.jpg   04009.jpg	      08007.jpg		 12005_1.jpg	    15993_1.jpg
 00002.jpg   04010.jpg	      08008.jpg		 12006_1.jpg	    15994_1.jpg
 00003.jpg   04011.jpg	      08009.jpg		 12007_1.jpg	    15995_1.jpg
 00004.jpg   04012.jpg	      08010.jpg		 12008_1.jpg	    15996_1.jpg
 00005.jpg   04013.jpg	      08011.jpg		 12009_1.jpg	    15997_1.jpg
 00006.jpg   04014.jpg	      08012.jpg		 12010_1.jpg	    15998_1.jpg
 00007.jpg   04015.jpg	      08013.jpg		 12011_1.jpg	    15999_1.jpg
 00008.jpg   04016.jpg	      08014.jpg		 12012_1.jpg	    16000_1.jpg
 00009.jpg   04017.jpg	      08015.jpg		 12013_1.jpg	    16001_1.jpg
 00010.jpg   04018.jpg	      08016.jpg		 12014_1.jpg	    16002_1.jpg
 00011.jpg   04019.jpg	      08017.jpg		 12015_1.jpg	    16003_1.jpg
 00012.jpg   04020.jpg	      08018.jpg		 12016_1.jpg	    16004_1.jpg
 00013.jpg   04021.jpg	      08019.jpg		 12017_1.jpg	    16005_1.jpg
 00014.jpg   04022.jpg	      08020.jpg		 12018_1.jpg	    16006_1.jpg
 00015.jpg   04023.jpg	      08021

In [ ]:
def generate_df(image_dir, label):
    filepaths = pd.Series(list(image_dir.glob(r'*.jpg')), name='Filepath').astype(str)
    labels = pd.Series(label, name='Label', index=filepaths.index)
    df = pd.concat([filepaths, labels], axis=1)
    return df

In [ ]:
positive_df = generate_df(positive_dir, label="POSITIVE")
negative_df = generate_df(negative_dir, label="NEGATIVE")

all_df = pd.concat([positive_df, negative_df], axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)
all_df

,Filepath,Label
0,/content/drive/My Drive/Positive/09423.jpg,POSITIVE
1,/content/drive/My Drive/Positive/14126_1.jpg,POSITIVE
2,/content/drive/My Drive/Positive/10494_1.jpg,POSITIVE
3,/content/drive/My Drive/Positive/00235.jpg,POSITIVE
4,/content/drive/My Drive/Negative/05089.jpg,NEGATIVE
...,...,...
40045,/content/drive/My Drive/Positive/10627_1.jpg,POSITIVE
40046,/content/drive/My Drive/Negative/15379.jpg,NEGATIVE
40047,/content/drive/My Drive/Positive/08826.jpg,POSITIVE
40048,/content/drive/My Drive/Positive/14336_1.jpg,POSITIVE


In [ ]:
list(positive_dir.glob(r'*.jpg'))

[PosixPath('/content/drive/My Drive/Positive/10680_1.jpg'),
 PosixPath('/content/drive/My Drive/Positive/13934_1.jpg'),
 PosixPath('/content/drive/My Drive/Positive/10922_1.jpg'),
 PosixPath('/content/drive/My Drive/Positive/19794.jpg'),
 PosixPath('/content/drive/My Drive/Positive/12715_1.jpg'),
 PosixPath('/content/drive/My Drive/Positive/00023.jpg'),
 PosixPath('/content/drive/My Drive/Positive/10746_1.jpg'),
 PosixPath('/content/drive/My Drive/Positive/00208.jpg'),
 PosixPath('/content/drive/My Drive/Positive/18283_1.jpg'),
 PosixPath('/content/drive/My Drive/Positive/19423.jpg'),
 PosixPath('/content/drive/My Drive/Positive/02520.jpg'),
 PosixPath('/content/drive/My Drive/Positive/01560.jpg'),
 PosixPath('/content/drive/My Drive/Positive/13257_1.jpg'),
 PosixPath('/content/drive/My Drive/Positive/02276.jpg'),
 PosixPath('/content/drive/My Drive/Positive/11712_1.jpg'),
 PosixPath('/content/drive/My Drive/Positive/09528.jpg'),
 PosixPath('/content/drive/My Drive/Positive/06875.jpg')

In [ ]:
train_df, test_df = train_test_split(
    all_df.sample(6000, random_state=1),
    train_size=0.7,
    shuffle= True,
    random_state=1
)

In [ ]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_data = train_gen.flow_from_dataframe(
    train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_data = train_gen.flow_from_dataframe(
    train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_data = train_gen.flow_from_dataframe(
    test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False,
    seed=42
)

Found 3360 validated image filenames belonging to 2 classes.
Found 840 validated image filenames belonging to 2 classes.
Found 1800 validated image filenames belonging to 2 classes.
